category class 개수 줄이는 코드임
임의로 car, person, bollard만 추출해봄 -> 차후 모두 같이 상의해서 추출할 class 정하기
car, person, bollard가 포함되어 있지 않은 이미지에 대해서는 빈 txt가 생성됨!!!!!!!!!!!!!

0822_all_raw_xml_to_tx_EJ.ipynb파일에서 아래 부분만 추가된 코드임
##########################################################
#특정 물체에 대한 box들만 가져오기
condition = (df['label'] == 'car') | (df['label'] == 'person') | (df['label'] == 'bollard')
df=df[condition]
##########################################################

In [1]:
import pandas as pd
import os
import glob

# xml파일 모두 불러오기
file_list=glob.glob('/Users/ejju/Desktop/개발자대회_WISET/ju/xml_files_ju/*.xml')
file_list=sorted(file_list)

# 모든 xml파일에서 데이터 가져오기
for file in file_list:
    import xml.etree.ElementTree as ET
    tree = ET.parse(file)
    root = tree.getroot()
    
    
    rows=[]

    for i in range(2,len(root)):
        id=root[i].attrib['id']
        name=root[i].attrib['name']
        x=root[i] #중요!!! 이전 파일에서 x를 선언해주지 않아서 x.attrib관련 수치들이 랜덤으로 들어왔었음
        for j in range(len(x)):
            label = x[j].attrib['label']
            xtl = x[j].attrib['xtl']
            ytl = x[j].attrib['ytl']
            xbr = x[j].attrib['xbr']
            ybr = x[j].attrib['ybr']
            rows.append({
                "id": id,
                "name": name,
                "label": label,
                "xtl": xtl,
                "ytl": ytl,
                "xbr": xbr,
                "ybr": ybr,
                })
    
    # 데이터프레임으로 변환했을 때 열(row) 수와 일치하는지 확인하기 위해 rows의 길이를 확인        
    #print(len(rows))
    # 데이터프레임으로 보기
    df = pd.DataFrame(rows)
    
    #특정 물체에 대한 box들만 가져오기
    condition = (df['label'] == 'car') | (df['label'] == 'person') | (df['label'] == 'bollard')
    df=df[condition]
    
    # 문자열로 표현되어 있는 좌표 표시 수치를 숫자형으로 변환-팀장님은 타입변환없이 진행한 거 보면 roboflow에서는 문자열로 표현된 숫자도 인식하는듯????
    #df[['xtl','ytl','xbr','ybr']]=df[['xtl','ytl','xbr','ybr']].apply(pd.to_numeric)

    # 여러 txt파일이 생성되므로 먼저 저장할 폴더 생성하기
    def createFolder(directory):
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print('Error:Creating directory.'+directory)
    createFolder('/Users/ejju/Desktop/개발자대회_WISET/ju/3classes_all_txt_files')

    # 위 셀에서 한 이미지파일에 대해 하나의 txt파일 만든 것을 반복문에 이용하여 모든 이미지파일에 적용하여 각각의 txt파일 만들기
    # 파일 이름들이 저장되는 리스트 만들기
    file_names=[]
    for i in range(2, len(root)):
        name=root[i].attrib['name']
        file_names.append(name)

    # 각 이미지파일에 해당하는 행들끼리 나눠주기
    df_txt_files=[]
    for i in range(len(file_names)):
        df_image=(df['name']==file_names[i])
        df_txt_file=df[df_image]
        df_txt_files.append(df_txt_file)

    # 각 txt파일에 들어갈 컬럼들만 추출    
    df_columns=[]
    for i in range(len(df_txt_files)):
        df_column=df_txt_files[i][['label','xtl','ytl','xbr','ybr']]
        df_columns.append(df_column)

    # 각 이미지이름과 동일하도록 txt파일 이름 정하고, 각 txt파일에 들어갈 컬럼들만 추출한 것으로 txt파일 만들기
    for i in range(len(file_names)):
        file_name=file_names[i].split('.')[0]
        df_columns[i].to_csv('/Users/ejju/Desktop/개발자대회_WISET/ju/3classes_all_txt_files/'+file_name+'.txt', sep=' ', index=False, header=None)